In [ ]:
import numpy as np
import pandas as pd
import regex as re
import random as rn
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,average_precision_score, precision_score,precision_recall_curve
from tqdm.notebook import tqdm
from tqdm import trange
import warnings
import pickle
import nltk
import math
import os
import json
import random
import re
import torch
import torch.nn as nn
import spacy 
from spacy import displacy
#import neptune.new as neptune                          
from torch.utils.data import (DataLoader, RandomSampler, WeightedRandomSampler, SequentialSampler, TensorDataset, Dataset)

In [ ]:
!pip install emoji -q
!pip install datasets -q
!pip install rouge_score -q


!pip install transformers -q
!pip install sentencepiece -q
!pip install pandarallel -q

from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel

from transformers import AdamW, get_linear_schedule_with_warmup,PegasusConfig, PegasusTokenizer, PegasusForConditionalGeneration
from transformers import AutoTokenizer, AutoModel, set_seed, TrainingArguments, Trainer, default_data_collator, AdamW, get_linear_schedule_with_warmup, Seq2SeqTrainer, Seq2SeqTrainingArguments


     |████████████████████████████████| 170 kB 4.3 MB/s 
     |████████████████████████████████| 298 kB 4.4 MB/s 
     |████████████████████████████████| 132 kB 48.4 MB/s 
     |████████████████████████████████| 61 kB 412 kB/s 
     |████████████████████████████████| 1.1 MB 29.3 MB/s 
     |████████████████████████████████| 243 kB 54.3 MB/s 
     |████████████████████████████████| 192 kB 37.6 MB/s 
     |████████████████████████████████| 160 kB 54.5 MB/s 
     |████████████████████████████████| 271 kB 34.4 MB/s 
     |████████████████████████████████| 3.4 MB 4.3 MB/s 
     |████████████████████████████████| 596 kB 51.9 MB/s 
     |████████████████████████████████| 895 kB 58.4 MB/s 
     |████████████████████████████████| 3.3 MB 41.7 MB/s 
     |████████████████████████████████| 1.2 MB 4.2 MB/s 


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [ ]:
ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
... A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
... Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
... In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
... Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
... 2010 marriage license application, according to court documents.
... Prosecutors said the marriages were part of an immigration scam.
... On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
... After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
... Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
... All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
... Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
... Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
... The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
... Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
... Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
... If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
... """

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Mounted at /content/drive


In [ ]:
reviews_path = '/content/drive/MyDrive/gen_datasets/hotel_travel_reviews.json'
# reviews_path = '/content/drive/MyDrive/restaurant_reviews/res_reviews_final.json'
import json
f = open(reviews_path)
# data = json.load(f)
data = json.load(f)
x = data

In [ ]:
# x = {}
# for key_ in data.keys():
#   for a in data[key_]['review_list']:
#     if(a['rating']!=None):
#       x[a['text']] = a['rating']

In [ ]:
sorted_vals = sorted(x.items(), key=lambda item: item[1])

In [ ]:
reviews = []
for j in sorted_vals:
  reviews.append(j[0])

In [ ]:
num_kp = int(len(reviews)/50)
input = []
for i in range(len(reviews)):
  if(i%num_kp==0):
    input.append(reviews[i])
    src_text = ''.join(input)
    batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    print(i,tgt_text)
    input = []

0 ["It's disgusting and way over priced. it's disgusting and way"]
71 ['it was a good place to stay i went to the front desk and the']
142 ['I was so pissed. I was in line for 1 hour and 15 min']
213 ['I was told I could wash the truck myself and they would reimburse me. a']
284 ["I'm not sure why Budget has such a long line. over 30 minutes"]
355 ['I was told I didnt prepay and that I was being offered smaller vehicles that didn']
426 ['I have a baby. I have a baby. I have a']
497 ["I booked this car way in advance. I'm fine.. I'"]
568 ['the room was very small. it was a good motel. I would']
639 ['I would say,. a million times...']
710 ['I would have to wait for the car to turn on.. I would have']
781 ['he was not there. We called Sebastian and he said he would call us']
852 ['i was excited to learn that BrewDog was opening up their brewery in Canal']
923 ['I was very disappointed. I was very disappointed. Caught a ringworm']
994 ['My alarm has been going off for over half an hour. t